In [1]:
from stdmodandoption import *
import time

runtodo='m12imhdcvhr'
i=580
info=SSF.outdirname(runtodo, i)
rundir=info['rundir']
Nsnapstring=info['Nsnapstring']
havecr=info['havecr']
haveB=info['haveB']
cutcold=0
dx=dy=dz=1
commonpath='/home/tkc004/scratch/snipshot/philruns/'
#SSF.mkdir_p(commonpath+rundir+'/output/withinr200spno100')
fname=commonpath+rundir+'/output/withinr200GS/snipshot_'+Nsnapstring+'.hdf5'
data = RSS.readsnipshot(fname,ptypelist = [0])

G = data[0];

rundir /m12i_mass7000_MHDCR_tkFIX/mhdcv/


In [2]:

def calgfromparlocal(G,pos,withinr,spno):
    data = SSF.calgfromparlist(G,pos=pos,withinr=withinr,spno=spno)
    gxl = data['gx']; gyl = data['gy']; gzl = data['gz'];
    return [gxl,gyl,gzl]

def calgfromparlocal1(G,pos,withinr,spno,i):
    data = SSF.calgfromparlist(G,pos=pos,withinr=withinr,spno=spno)
    gxl = data['gx']; gyl = data['gy']; gzl = data['gz'];
    return [[i,gxl,gyl,gzl]]

def calgfromparlocal3(Glist,pos,withinr,spno,i):
    gxl = np.zeros(len(pos)); gyl = np.zeros(len(pos)); gzl = np.zeros(len(pos))
    for G in Glist:
        data = SSF.calgfromparlist(G,pos=pos,withinr=withinr,spno=spno)
        gxl += data['gx']; gyl += data['gy']; gzl += data['gz'];
    return [[i,gxl,gyl,gzl]]

def pressureXYZlocal(G, pos, dx, dy, dz,havecr,haveB,cutcold,i):
    data = CRTF.pressureXYZ(G, pos, dx, dy, dz,havecr=havecr,haveB=haveB,cutcold=cutcold)
    return [[i,data]]

def calgfromparlocal2(G,withinr,spno,pos):
    data = SSF.calgfromparlist(G,pos=pos,withinr=withinr,spno=spno)
    gxl = data['gx']; gyl = data['gy']; gzl = data['gz'];
    return [gxl,gyl,gzl]

def chunks(l, n):
    n = max(1, n)
    lenl = len(l)
    chucklist = [l[i:i+n] for i in xrange(0, lenl, n)]
    return chucklist
        
def collect_results(result):
    results.extend(result)


In [3]:
maxlength=20
nogrid=15

zmax=maxlength/2.0
xlist = ylist = zlist = np.linspace(-zmax,zmax,num=nogrid)
xl,yl,zl = np.meshgrid(xlist,ylist,zlist)
xl = np.ravel(xl); yl = np.ravel(yl); zl = np.ravel(zl);
gxl=np.array([]); gyl=np.array([]); gzl=np.array([]);

In [11]:
import multiprocessing as mp
pos=[]
for x,y,z in zip(xl,yl,zl):
    pos.append([x,y,z])

nocpu = 16

print 'nocpu', nocpu

lenpos = len(pos)

chunksize = lenpos/nocpu

listpos = chunks(pos, chunksize)

nocpu 16


In [10]:
start = time.time()
dendata = CRTF.pressureXYZ(G, pos, dx, dy, dz,havecr=havecr,haveB=haveB,cutcold=cutcold)
end = time.time()
print(end - start)

432.020654917


In [12]:
import multiprocessing as mp

start = time.time()


withinr=200; spno=1
# Step 1: Init multiprocessing.Pool()


pool = mp.Pool(nocpu)
results=[]

# Step 2: `pool.apply` the `howmany_within_range()
#Ggxyz = [pool.map(calgfromparpart, pos)]
#Sgxyz = [pool.map(calgfromparpart, pos)]
#DMgxyz = [pool.map(calgfromparpart, pos)]
pxyz = [pool.apply_async(pressureXYZlocal, args=(G, xyz, dx, dy, dz,havecr,haveB,cutcold,i), callback=collect_results)  for i, xyz in enumerate(listpos)]
#gxyz = [pool.apply_async(calgfromparlocal3, args=([G,S,DM],xyz,withinr,spno,i), callback=collect_results)  for i, xyz in enumerate(listpos)]
#Ggxyz = [pool.apply_async(calgfromparlocal1, args=(G,xyz,withinr,spno,i), callback=collect_results)  for i, xyz in enumerate(listpos)]
#Sgxyz = [pool.apply_async(calgfromparlocal1, args=(S,xyz,withinr,spno,i), callback=collect_results) for i, xyz in enumerate(listpos)]
#DMgxyz = [pool.apply_async(calgfromparlocal1, args=(DM,xyz,withinr,spno,i), callback=collect_results) for i, xyz in enumerate(listpos)]
#Ggxyz = [pool.apply(calgfromparlocal, args=(G,xyz,withinr,spno)) for xyz in listpos]
#Sgxyz = [pool.apply(calgfromparlocal, args=(S,xyz,withinr,spno)) for xyz in listpos]
#DMgxyz = [pool.apply(calgfromparlocal, args=(DM,xyz,withinr,spno)) for xyz in listpos]
# Step 3: Don't forget to close
pool.close()
pool.join()

end = time.time()
print(end - start)

92.8036231995


In [210]:
argsortlist=np.argsort([results[i][0] for i in range(len(results))])
gxlist =np.array([results[i][1] for i in range(len(results))])
gxlnew = gxlist[argsortlist].flatten()
gylist =np.array([results[i][2] for i in range(len(results))])
gylnew = gylist[argsortlist].flatten()
gzlist =np.array([results[i][3] for i in range(len(results))])
gzlnew = gzlist[argsortlist].flatten()

In [8]:
results

[[0,
  {'kezl': array([9.66541163e+50, 8.80039912e+50, 6.46178579e+50, 3.74787780e+48,
          1.15066315e+51, 2.05764796e+51, 4.53821477e+51, 0.00000000e+00,
          1.33721414e+51, 2.30699419e+51, 2.47241498e+51, 0.00000000e+00,
          9.76708852e+50, 4.78188476e+50, 5.19437062e+51, 2.80733118e+51,
          0.00000000e+00, 8.69007435e+50, 1.67811274e+51, 9.05395220e+50,
          7.67970037e+50, 7.91768661e+50, 1.65987423e+51, 4.70316415e+51,
          3.95799624e+51, 2.83847183e+51, 6.45220551e+51, 1.07910545e+52,
          0.00000000e+00, 4.32128611e+50, 3.62246661e+50, 1.79868940e+50,
          6.88306870e+50, 4.70602798e+51, 1.31613205e+51, 1.12313099e+51,
          9.46665404e+50, 2.09896277e+51, 1.75011677e+51, 7.74005898e+50,
          4.05444658e+50, 1.43318008e+50, 1.29010398e+52, 4.94700084e+51,
          2.62581454e+51, 8.77045422e+48, 7.40653096e+50, 2.40841528e+51,
          2.37860477e+51, 8.40699390e+51, 7.16328747e+50, 1.67331132e+50,
          1.14329434e+51,

In [31]:
def joindata(results):
    argsortlist=np.argsort([results[i][0] for i in range(len(results))])
    totdict = np.array([results[i][1] for i in range(len(results))])
    totdict=totdict[argsortlist]
    comdict={}
    for key in totdict[0]:
        totarray=np.array([])
        for indict in totdict:
            totarray = np.append(totarray,indict[key])
        comdict[key] = totarray.flatten()
    return comdict

In [32]:
comdict=joindata(results)

In [33]:
dendata['kezl']

array([9.66541163e+50, 0.00000000e+00, 7.87142212e+50, ...,
       6.08293360e+52, 2.03462138e+52, 9.64362331e+50])

In [34]:
comdict['kezl']-dendata['kezl']

array([0., 0., 0., ..., 0., 0., 0.])